Denoising using GRU layers never got me anywhere, but I'm leaving it for the future as a potential inspiration

In [1]:
import numpy as np
import pandas as pd

from tensorflow import keras

In [2]:
train_raw = pd.read_csv('data/train.csv').drop('id', axis=1).to_numpy()
split = int(train_raw.shape[0] * .6)

In [3]:
def prepare_for_rnn(data, n_in=1, n_out=1):
    n_vars = data.shape[1]
    df = pd.DataFrame(data)
    cols, names = [], []

    for i in range(n_in, 0, -1):
        shifted = df.shift(i).fillna(0)

        cols.append(shifted)

        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]

    for i in range(0, n_out):
        shifted = df.shift(-i).fillna(0)

        cols.append(shifted)

        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names

    return agg


train_rnn = prepare_for_rnn(train_raw, 1, 3)
values = train_rnn.values
train_rnn = values[:split, :]
test_rnn = values[split:, :]
valid_rnn, test_rnn = np.array_split(test_rnn, 2)
X_train_rnn, y_train_rnn = train_rnn[:, :-1], train_rnn[:, -1]
X_valid_rnn, y_valid_rnn = valid_rnn[:, :-1], valid_rnn[:, -1]
X_test_rnn, y_test_rnn = test_rnn[:, :-1], test_rnn[:, -1]
X_train_rnn = X_train_rnn.reshape((X_train_rnn.shape[0], 1, X_train_rnn.shape[1]))
X_valid_rnn = X_valid_rnn.reshape((X_valid_rnn.shape[0], 1, X_valid_rnn.shape[1]))
X_test_rnn = X_test_rnn.reshape((X_test_rnn.shape[0], 1, X_test_rnn.shape[1]))

In [8]:
encoder = keras.models.Sequential([
    keras.layers.GRU(59, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    keras.layers.GaussianNoise(.02),
    keras.layers.Dense(14, activation='relu')
])
decoder = keras.models.Sequential([
    keras.layers.Dense(14, activation='relu'),
    keras.layers.RepeatVector(3),
    keras.layers.GRU(59)
])
denoising_ae = keras.models.Sequential([encoder, decoder])

encoder.summary()
decoder.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_4 (GRU)                 (None, 59)                21240     
                                                                 
 gaussian_noise_2 (GaussianN  (None, 59)               0         
 oise)                                                           
                                                                 
 dense_4 (Dense)             (None, 14)                840       
                                                                 
Total params: 22,080
Trainable params: 22,080
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 14)                210       
                            

In [ ]:
denoising_ae.compile(optimizer='adam', loss='mse')
denoising_ae.fit(X_train_rnn, X_train_rnn, epochs=50, batch_size=128,
                 validation_data=(X_valid_rnn, X_valid_rnn))